In [130]:
import pandas as pd
import numpy as np
from jqdatasdk import *
import grequests
import requests
import json

auth('13810650842','123456a')
pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [133]:
df_r = pd.read_excel('all.xlsx', index_col=0, dtype={'code':str})
# df_r = df_r[df_r['code'] == 100032]
# df_r.code = df_r.code.apply(lambda x: str(x).zfill(6))
df_r = df_r[~df_r.account.isin(['华泰','华宝','康力泉_蛋卷'])]

df_nav = pd.read_excel('nav.xlsx', index_col=0, dtype={'code':str})
df_nav.date = df_nav.date.astype(np.datetime64)
# df_nav.code = df_r.code.apply(lambda x: str(x).zfill(6))
# df_nav = df_nav[df_nav['code'] == 100032]
# df_nav

# df_day = pd.read_excel('days.xlsx', index_col=0)
# df_day

In [134]:

def calc_date(x):
    op = x['deal_type']
    if op == '分红':
        # 日期游标（除权日，可能在给定日期前后的一个范围内）
        # 非货币基金的分红，通常稍有一个月触发 2 次及以上的
        date_cursor = x['date']
        # 上边界，15 天以前
        before = pd.DatetimeIndex(end=date_cursor, freq='D', periods=15)[0]
        # 下边界 15 天以后
        after = pd.DatetimeIndex(start=date_cursor, freq='D', periods=15)[-1]
        
        df_temp = df_nav[(df_nav['code'] == x['code']) & (df_nav['date'] >= before) & (df_nav['date'] <= after) & (df_nav['deal_type'] == '分红')]
#         print(date_cursor, before, after, df_temp)
        if len(df_temp) > 0:
            return str(df_temp.date.values[0])[0:10]
        else:
            return '无法补充'
        pass
    else:
        return x['date']
    pass

def calc_nav_unit(x):
    df_temp = df_nav[(df_nav['code'] == x['code']) & (df_nav['date'] == x['补充日期'])]
#     print(df_temp)
    if len(df_temp) > 0:
        return df_temp.nav_unit.values[0]
    else:
#         return '无法补充'
        return x['nav_unit']

def calc_nav_acc(x):
    df_temp = df_nav[(df_nav['code'] == x['code']) & (df_nav['date'] == x['补充日期'])]
#     print(df_temp)
    if len(df_temp) > 0:
        return df_temp.nav_acc.values[0]
    else:
#         return '无法补充'
        return x['nav_acc']

df = df_r
df['补充日期'] = df['date']
df['补充日期'] = df.apply(calc_date, axis=1)
df['单位净值'] = df.apply(calc_nav_unit, axis=1)
df['累计净值'] = df.apply(calc_nav_acc, axis=1)
df['nav_比较'] = df['nav_unit'] == df['单位净值']
# df['acc_比较'] = df['nav_acc'] == df['累计净值']
df = df[['code','name', 'deal_type', 'date', '补充日期','nav_unit','单位净值','nav_比较', 'nav_acc','累计净值', "account"]]
df[df['nav_比较'] == False]

D:\develop\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  
D:\develop\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  # Remove the CWD from sys.path while we load stuff.


,code,name,deal_type,date,补充日期,nav_unit,单位净值,nav_比较,nav_acc,累计净值,account
238,001061,华夏收益债券(QDII)A,卖出,2018-10-31,2018-10-31,1.2570,1.2550,False,1.2570,1.4190,康力泉_天天
242,164906,交银中证海外中国互联网指数,买入,2018-11-16,2018-11-16,1.1580,1.1500,False,1.1580,1.1500,康力泉_天天
247,000614,华安德国30(DAX)联接,买入,2018-12-03,2018-12-03,1.0790,1.0910,False,1.0790,1.0910,康力泉_天天
248,162411,华宝油气,买入,2018-12-03,2018-12-03,0.5670,0.5860,False,0.5670,0.5860,康力泉_天天
252,162411,华宝油气,买入,2018-12-28,2018-12-28,0.4670,0.4580,False,0.4670,0.4580,康力泉_天天
267,000614,华安德国30(DAX)联接,买入,2019-03-29,2019-03-29,1.0520,1.0590,False,1.0520,1.0590,康力泉_天天
268,162411,华宝油气,买入,2019-03-29,2019-03-29,0.5190,0.5180,False,0.5190,0.5180,康力泉_天天
330,164906,交银中证海外中国互联网指数,买入,2019-05-31,2019-05-31,1.1140,1.1040,False,1.1140,1.1040,康力泉_天天
376,001061,华夏收益债券(QDII)A,分红,2019-07-10,2019-07-10,0.0000,1.3040,False,0.0000,1.5060,康力泉_天天
719,501018,南方原油,买入,2020-03-10,2020-03-10,0.7162,0.7310,False,0.7162,0.7310,康力泉_天天
